## Portfolio Exercise Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their candidates.   The data for this exercise consists of about 120,000 data points split in 2:1 ratio among training and test files. Each data point includes one column indicating whether or not an individual was sent a promotion for a specific product, and one column indicating whether or not that individual eventually purchased that product. Each individual also had seven additional features associated with them.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user.  Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

Ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group.

* **Net Incremental Revenue (NIR)**

The total number of purchasers that received the promotion times 10 minus the number of promotions given times 0.15 minus the number of purchasers who were not given the promotion times 10.

For a full description of what starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the **promotion_strategy** function to pass to the **test_results** function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers.  

|   | Actual      |    |    | 
|---|-------------|----|----|
| **Predicted**   | Yes| No |  
| Yes             | **I**   | **II**|  
| No              | **III** | **IV**|  

The metrics are only being compared for the individual's we predict should obtain the promotion - that is quadrants I and II here.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equal participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 


Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final test_results function.

In [1]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# load in the data
train_data = pd.read_csv('data/training.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


* **Incremental Response Rate (IRR)** 

Ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group.

In [2]:
yes_df, no_df = train_data[train_data.Promotion.str.contains('Yes')], train_data[train_data.Promotion.str.contains('No')]  

In [12]:
y_promo, y_no = yes_df.purchase.sum(), no_df.purchase.sum()

m_promo, m_no = len(yes_df), len(no_df)

iir = y_promo / m_promo - y_no / m_no
print(f'IIR: {iir:.5f}')

IIR: 0.00945


* **Net Incremental Revenue (NIR)**

The total number of purchasers that received the promotion times 10 minus the number of promotions given times 0.15 minus the number of purchasers who were not given the promotion times 10.

In [10]:
nir = y_promo * 10 - m_promo * .15 - y_no * 10
print(f'NIR: {nir:.2f}')

NIR: -2334.60


In [29]:
def nir(y_promo, m_promo, y_no):
    return y_promo * 10 - m_promo * .15 - y_no * 10

In [13]:
train_data.corr()
    

,ID,purchase,V1,V2,V3,V4,V5,V6,V7
ID,1.000000,0.006902,-0.004199,-0.002757,0.003296,-0.010600,0.000133,-0.001434,0.001686
purchase,0.006902,1.000000,-0.004906,-0.001694,-0.006395,0.032245,0.007814,-0.001538,-0.001117
V1,-0.004199,-0.004906,1.000000,-0.001564,0.003995,0.002568,-0.001196,-0.003672,0.001436
V2,-0.002757,-0.001694,-0.001564,1.000000,0.001165,0.002848,-0.001046,-0.004156,-0.002293
V3,0.003296,-0.006395,0.003995,0.001165,1.000000,0.003753,-0.001736,0.004464,0.001135
V4,-0.010600,0.032245,0.002568,0.002848,0.003753,1.000000,0.003161,0.002552,-0.003176
V5,0.000133,0.007814,-0.001196,-0.001046,-0.001736,0.003161,1.000000,-0.003043,0.003571
V6,-0.001434,-0.001538,-0.003672,-0.004156,0.004464,0.002552,-0.003043,1.000000,-0.001398
V7,0.001686,-0.001117,0.001436,-0.002293,0.001135,-0.003176,0.003571,-0.001398,1.000000


In [14]:
yes_df.corr()

,ID,purchase,V1,V2,V3,V4,V5,V6,V7
ID,1.000000,0.005549,-0.002255,-0.002680,0.001301,-0.012418,-0.000783,-0.003812,-0.001706
purchase,0.005549,1.000000,-0.004651,-0.002885,-0.017012,0.053348,0.011711,-0.002049,-0.005005
V1,-0.002255,-0.004651,1.000000,-0.001973,0.009417,0.001857,-0.005020,-0.007736,0.002503
V2,-0.002680,-0.002885,-0.001973,1.000000,-0.002440,-0.001763,-0.003660,-0.002544,-0.004795
V3,0.001301,-0.017012,0.009417,-0.002440,1.000000,-0.000656,-0.001973,0.007905,-0.000368
V4,-0.012418,0.053348,0.001857,-0.001763,-0.000656,1.000000,0.000586,-0.002490,-0.003113
V5,-0.000783,0.011711,-0.005020,-0.003660,-0.001973,0.000586,1.000000,0.001858,0.002265
V6,-0.003812,-0.002049,-0.007736,-0.002544,0.007905,-0.002490,0.001858,1.000000,-0.004016
V7,-0.001706,-0.005005,0.002503,-0.004795,-0.000368,-0.003113,0.002265,-0.004016,1.000000


In [15]:
no_df.corr()

,ID,purchase,V1,V2,V3,V4,V5,V6,V7
ID,1.000000,0.009648,-0.006105,-0.002838,0.005327,-0.008794,0.001021,0.000948,0.005067
purchase,0.009648,1.000000,-0.006088,0.000090,0.008325,0.002344,0.002771,-0.000980,0.004786
V1,-0.006105,-0.006088,1.000000,-0.001145,-0.001499,0.003297,0.002673,0.000397,0.000373
V2,-0.002838,0.000090,-0.001145,1.000000,0.004798,0.007508,0.001585,-0.005784,0.000235
V3,0.005327,0.008325,-0.001499,0.004798,1.000000,0.008187,-0.001448,0.001007,0.002656
V4,-0.008794,0.002344,0.003297,0.007508,0.008187,1.000000,0.005736,0.007627,-0.003243
V5,0.001021,0.002771,0.002673,0.001585,-0.001448,0.005736,1.000000,-0.007947,0.004874
V6,0.000948,-0.000980,0.000397,-0.005784,0.001007,0.007627,-0.007947,1.000000,0.001237
V7,0.005067,0.004786,0.000373,0.000235,0.002656,-0.003243,0.004874,0.001237,1.000000


In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [20]:
train_data.Promotion = train_data.Promotion.apply(lambda x: 1 if x == 'Yes' else 0)
train_data

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,0,0,2,30.443518,-1.165083,1,1,3,2
1,3,0,0,3,32.159350,-0.645617,2,3,2,2
2,4,0,0,2,30.431659,0.133583,1,1,4,2
3,5,0,0,0,26.588914,-0.212728,2,1,4,2
4,8,1,0,3,28.044332,-0.385883,1,1,2,2
5,9,0,0,1,41.189415,-0.905350,2,4,4,1
6,11,0,0,1,40.690409,1.085939,2,2,1,2
7,14,0,0,2,25.971529,-1.424817,1,3,3,2
8,15,1,0,2,31.930423,0.393317,2,3,1,2
9,16,0,0,1,27.474650,0.566472,2,2,3,2


In [21]:

x_train, y = train_data.drop(columns=['ID', 'purchase']), train_data['purchase']

In [22]:
rfc = RandomForestClassifier()
rfc.fit(x_train, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [24]:
for c, f in zip(x_train.columns, rfc.feature_importances_):
    print(f'{c}: {f}')

Promotion: 0.007071947793844895
V1: 0.03488903088018125
V2: 0.6943162141850772
V3: 0.16990204452104624
V4: 0.0048398641975992375
V5: 0.030404021573697444
V6: 0.04647387186070568
V7: 0.012103004987848023


In [25]:
test_data = pd.read_csv('data/Test.csv')

In [26]:
test_data.Promotion = test_data.Promotion.apply(lambda x: 1 if x == 'Yes' else 0)

In [27]:
x_test, y_test = test_data.drop(columns=['ID', 'purchase']), test_data['purchase']

In [31]:
pred = rfc.predict(x_test)
print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))

             precision    recall  f1-score   support

          0       0.99      1.00      0.99     41170
          1       0.03      0.00      0.00       480

avg / total       0.98      0.99      0.98     41650

[[41131    39]
 [  479     1]]


In [ ]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    
    
    
    
    return promotion

In [ ]:
# This will test your results, and provide you back some information 
# on your how well your promotion_strategy will work in practice

test_results(promotion_strategy)